In [ ]:
import tensorflow as tf
import numpy as np
import os
import boto3

In [ ]:
nodes_number = 32
learning_rate = 0.0001

## Train a model

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

In [ ]:
def filter_36(x, y):
    keep = (y == 3) | (y == 6)
    x, y = x[keep], y[keep]
    y = y == 3
    return x,y

print("Number of unfiltered training examples:", len(x_train))
print("Number of unfiltered test examples:", len(x_test))

x_train, y_train = filter_36(x_train, y_train)
x_test, y_test = filter_36(x_test, y_test)

print("Number of filtered training examples:", len(x_train))
print("Number of filtered test examples:", len(x_test))

In [ ]:
model = tf.keras.models.Sequential()

In [ ]:
model.add(tf.keras.layers.Conv2D(32, [3, 3], activation='relu', input_shape=(28,28,1)))
model.add(tf.keras.layers.Conv2D(64, [3, 3], activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(nodes_number, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))

In [ ]:
model.compile(loss='binary_crossentropy', \
              optimizer=tf.keras.optimizers.Adam(learning_rate), \
              metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_train, y_train, batch_size=256, epochs=1, verbose=1, validation_data=(x_test, y_test))
model_test_acc = model.evaluate(x_test, y_test)[1]

In [ ]:
print(model_test_acc)

## Store the model locally

In [ ]:
model_save_path = 'custom_model'
model.save(model_save_path)

## Store the model in an s3.cern.ch bucket

In [ ]:
os.environ['S3_ENDPOINT'] = 'https://s3.cern.ch'
client = boto3.client('s3', endpoint_url='https://s3.cern.ch')

bucket = 'BUCKET'

for root, dirs, files in os.walk(model_save_path):
    for file in files:
        name = os.path.join(root, file)[len(model_save_path) + 1:]
        bucket_object = 'custom_model/1/' + name
        client.upload_file(os.path.join(root, file), bucket, bucket_object)

## Make sure the model is properly stored in the bucket

In [ ]:
for key in client.list_objects(Bucket=bucket)['Contents']:
    object_name = key['Key']
    if 'custom_model' in object_name:
        print(object_name)